#### 一、 数据集加载

In [4]:
import findspark
findspark.init()
from pyspark import SparkContext, SQLContext, HiveContext, SparkConf
#sc.stop()
sc = SparkContext('local', 'pyspark')
sqlContext = HiveContext(sc)

In [5]:
data_dir = 'D:/python/Jupyter_Last_project/dataset/'

In [9]:
import pandas as pd
def load_movies_dataset():
    #movie_data_columns = [
    #'type_douban', 'actor', 'region', 'director', 'characteristic',
    #'score_douban', 'moviename']
    movie_data = pd.read_csv(data_dir + './dataset1/movie.csv')
    #movie_data.columns = movie_data_columns 
    return movie_data 

def load_user_and_ratings() :
    user_data = pd.read_csv(data_dir + './dataset1/user.csv')
    user_data['comment_time'] = pd.to_datetime(user_data['评论时间'])
    return user_data 

def load_doubanmovieshortcomments() :
    movieshortcomments_data = pd.read_csv(data_dir_doubanmovieshortcomments + './DMSC.csv')

    return movieshortcomments_data 
data_dir = 'D:/python/Jupyter_Last_project/dataset/'
data_dir_doubanmovieshortcomments = 'D:/python/Jupyter_Last_project/dataset/doubanmovieshortcomments/'
movies_data = load_movies_dataset()
print('movies_data.head()\n',movies_data.head())
user_data = load_user_and_ratings() 
#print('user_data.head()\n',user_data.head())
#movieshortcomments_data = load_doubanmovieshortcomments()
#print('movieshortcomments_data.head()\n',movieshortcomments_data.head())

movies_data.head()
    类型                  主演    地区     导演  特色   评分     电影名
0  剧情    徐峥|王传君|周一围|谭卓|章宇  中国大陆    文牧野  经典  8.9   我不是药神
1  剧情   冯小刚|许晴|张涵予|刘桦|李易峰  中国大陆     管虎  经典  7.8     老炮儿
2  剧情  王宝强|刘昊然|肖央|刘承羽|尚语贤  中国大陆    陈思诚  经典  6.7  唐人街探案2
3  剧情  任素汐|大力|刘帅良|裴魁山|阿如那  中国大陆  周申|刘露  经典  8.3     驴得水
4  剧情   徐峥|王宝强|李曼|李小璐|左小青  中国大陆    叶伟民  经典  7.5    人在囧途


In [28]:
from pyspark.sql.types import *
fields = [StructField("类型", StringType(), True),
       StructField("主演", StringType(), True),
       StructField("地区", StringType(), True),
       StructField("导演", StringType(), True),
       StructField("特色", StringType(), True),
       StructField("豆瓣评分", DoubleType(), True),
       StructField("电影名", StringType(), True)]

In [29]:
movie_spark_df = sqlContext.read.format("com.databricks.spark.csv")            \
        .options(delimiter=',')                                         \
        .options( header=True)                                          \
        .load(data_dir + './dataset1/movie.csv',schema = StructType(fields))

movie_spark_df.cache()
movie_spark_df.show( 5 )

+----+--------------------------------+--------+---------+----+--------+-----------+
|类型|                            主演|    地区|     导演|特色|豆瓣评分|     电影名|
+----+--------------------------------+--------+---------+----+--------+-----------+
|剧情|    徐峥|王传君|周一围|谭卓|章宇|中国大陆|   文牧野|经典|     8.9| 我不是药神|
|剧情|  冯小刚|许晴|张涵予|刘桦|李易峰|中国大陆|     管虎|经典|     7.8|     老炮儿|
|剧情|王宝强|刘昊然|肖央|刘承羽|尚语贤|中国大陆|   陈思诚|经典|     6.7|唐人街探案2|
|剧情|任素汐|大力|刘帅良|裴魁山|阿如那|中国大陆|周申|刘露|经典|     8.3|     驴得水|
|剧情|  徐峥|王宝强|李曼|李小璐|左小青|中国大陆|   叶伟民|经典|     7.5|   人在囧途|
+----+--------------------------------+--------+---------+----+--------+-----------+
only showing top 5 rows



In [30]:
print('user_data.head()\n',user_data.head())

user_data.head()
    评分     用户名                 评论时间  用户ID    电影名  类型        comment_time
0   2      身似  2018-01-05 15:05:06     1   心雨花露  爱情 2018-01-05 15:05:06
1   4  有意识的贱民  2018-01-05 15:05:06     3  战争的恐怖  战争 2018-01-05 15:05:06
2   2    亿万露电  2018-01-05 15:05:06     4  豪勇七蛟龙  战争 2018-01-05 15:05:06
3   2   Marni  2018-01-05 15:05:06     5   无序之主  犯罪 2018-01-05 15:05:06
4   4   马西嘻嘻嘻  2018-01-05 15:05:06     6  时装店风波  同性 2018-01-05 15:05:06


In [31]:
fields_user_rating = [StructField("用户评分", IntegerType(), True),
       StructField("用户名", StringType(), True),
       StructField("评论时间 ", DateType(), True),
       StructField("用户ID", StringType(), True),
       StructField("电影名", StringType(), True),
       StructField("类型", StringType(), True),]

In [32]:
user_ratings_spark_df = sqlContext.read.format("com.databricks.spark.csv")            \
        .options(delimiter=',')                                         \
        .options( header=True)                                          \
        .load(data_dir + './dataset1/user.csv',schema = StructType(fields_user_rating))

user_ratings_spark_df.cache()
user_ratings_spark_df.show( 5 )

+--------+------------+----------+------+----------+----+
|用户评分|      用户名| 评论时间 |用户ID|    电影名|类型|
+--------+------------+----------+------+----------+----+
|       2|        身似|2018-01-05|     1|  心雨花露|爱情|
|       4|有意识的贱民|2018-01-05|     3|战争的恐怖|战争|
|       2|    亿万露电|2018-01-05|     4|豪勇七蛟龙|战争|
|       2|       Marni|2018-01-05|     5|  无序之主|犯罪|
|       4|  马西嘻嘻嘻|2018-01-05|     6|时装店风波|同性|
+--------+------------+----------+------+----------+----+
only showing top 5 rows



In [33]:
user_movies_df = user_ratings_spark_df.join( movie_spark_df, ["电影名","类型"] , how = "inner" )
user_movies_df.show()

+----------------+----+--------+------------+----------+------+--------------------------+--------+---------------+----+--------+
|          电影名|类型|用户评分|      用户名| 评论时间 |用户ID|                      主演|    地区|           导演|特色|豆瓣评分|
+----------------+----+--------+------------+----------+------+--------------------------+--------+---------------+----+--------+
|            1028|灾难|       8|      对儿勾|2018-01-05|  1576|黄潇|刘翀|杨五洲|王默|肖睿|    美国|         闫海笑|青春|     8.8|
|            1028|灾难|       8|        Lynn|2018-01-08|  8832|黄潇|刘翀|杨五洲|王默|肖睿|    美国|         闫海笑|青春|     8.8|
|            1028|灾难|       4|        眠歌|2018-01-18| 25856|黄潇|刘翀|杨五洲|王默|肖睿|    美国|         闫海笑|青春|     8.8|
|            1028|灾难|       8|       Viola|2018-01-19| 15894|黄潇|刘翀|杨五洲|王默|肖睿|    美国|         闫海笑|青春|     8.8|
|            1028|灾难|       8|        散井|2018-01-19|  5056|黄潇|刘翀|杨五洲|王默|肖睿|    美国|         闫海笑|青春|     8.8|
|            1028|灾难|       4|潜水教练疯子|2018-01-23| 13239|黄潇|刘翀|杨五洲|王默|肖睿|    美国|         闫海笑|青春|   

In [34]:
user_movies_df.count()

239139

### 类别型特征和连续型特征

In [37]:
cat_features = ['类型',
              '地区',
              '导演',
              '特色']
continuous_features = ['豆瓣评分']


In [38]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, PolynomialExpansion, VectorIndexer
def create_category_vars( dataset, field_name ):
    #print('field_name\n',field_name)
    idx_col = field_name + "Index"
    col_vec = field_name + "Vec"
    #print('idx_col\n',idx_col)
    #print('col_vec\n',col_vec)
    #dataset = dataset.withColumn( idx_col,dataset[field_name] )
    #dataset = dataset.withColumn( col_vec,dataset[field_name] )
    print('dataset.columns\n',dataset.columns)
    month_stringIndexer = StringIndexer( inputCol=field_name,
                                       outputCol=idx_col )

    month_model = month_stringIndexer.fit( dataset )
    month_indexed = month_model.transform( dataset )

    month_encoder = OneHotEncoder( dropLast=True,
                                 inputCol=idx_col,
                                 outputCol= col_vec )

    return month_encoder.transform( month_indexed )

In [39]:
for col in cat_features:
    user_movies_df = create_category_vars( user_movies_df, col )
    user_movies_df.cache()

dataset.columns
 ['电影名', '类型', '用户评分', '用户名', '评论时间 ', '用户ID', '主演', '地区', '导演', '特色', '豆瓣评分']
dataset.columns
 ['电影名', '类型', '用户评分', '用户名', '评论时间 ', '用户ID', '主演', '地区', '导演', '特色', '豆瓣评分', '类型Index', '类型Vec']
dataset.columns
 ['电影名', '类型', '用户评分', '用户名', '评论时间 ', '用户ID', '主演', '地区', '导演', '特色', '豆瓣评分', '类型Index', '类型Vec', '地区Index', '地区Vec']
dataset.columns
 ['电影名', '类型', '用户评分', '用户名', '评论时间 ', '用户ID', '主演', '地区', '导演', '特色', '豆瓣评分', '类型Index', '类型Vec', '地区Index', '地区Vec', '导演Index', '导演Vec']


In [40]:
user_movies_df.show(2)

+------+----+--------+------+----------+------+--------------------------+----+------+----+--------+---------+---------------+---------+--------------+---------+--------------------+---------+-------------+
|电影名|类型|用户评分|用户名| 评论时间 |用户ID|                      主演|地区|  导演|特色|豆瓣评分|类型Index|        类型Vec|地区Index|       地区Vec|导演Index|             导演Vec|特色Index|      特色Vec|
+------+----+--------+------+----------+------+--------------------------+----+------+----+--------+---------+---------------+---------+--------------+---------+--------------------+---------+-------------+
|  1028|灾难|       8|对儿勾|2018-01-05|  1576|黄潇|刘翀|杨五洲|王默|肖睿|美国|闫海笑|青春|     8.8|     17.0|(18,[17],[1.0])|      0.0|(20,[0],[1.0])|   4529.0|(12155,[4529],[1.0])|      0.0|(8,[0],[1.0])|
|  1028|灾难|       8|  Lynn|2018-01-08|  8832|黄潇|刘翀|杨五洲|王默|肖睿|美国|闫海笑|青春|     8.8|     17.0|(18,[17],[1.0])|      0.0|(20,[0],[1.0])|   4529.0|(12155,[4529],[1.0])|      0.0|(8,[0],[1.0])|
+------+----+--------+------+----------+------+-------

In [41]:
categorical_vecs = [ "".join( (cat, "Vec") ) for cat in cat_features ]
categorical_vecs

['类型Vec', '地区Vec', '导演Vec', '特色Vec']

### 所有特征

In [42]:
all_features = continuous_features + categorical_vecs
all_features

['豆瓣评分', '类型Vec', '地区Vec', '导演Vec', '特色Vec']

#### 创建向量

In [43]:
assembler = VectorAssembler( inputCols = all_features, outputCol = "features")
user_movies_df = assembler.transform( user_movies_df )

In [44]:
user_movies_df.columns

['电影名',
 '类型',
 '用户评分',
 '用户名',
 '评论时间 ',
 '用户ID',
 '主演',
 '地区',
 '导演',
 '特色',
 '豆瓣评分',
 '类型Index',
 '类型Vec',
 '地区Index',
 '地区Vec',
 '导演Index',
 '导演Vec',
 '特色Index',
 '特色Vec',
 'features']

#### 预测目标变量——销量

In [45]:
user_movies_df = user_movies_df.withColumn( "label", user_movies_df.用户评分.cast( 'double' ) )
user_movies_df.select( "features", "label" ).show( 5 )

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(12202,[0,18,19,4...|  8.0|
|(12202,[0,18,19,4...|  8.0|
|(12202,[0,18,19,4...|  4.0|
|(12202,[0,18,19,4...|  8.0|
|(12202,[0,18,19,4...|  8.0|
+--------------------+-----+
only showing top 5 rows



In [53]:
user_movies_df = user_movies_df.drop("用户评分" )

In [55]:
user_movies_df.show(3)

+------+----+------+----------+------+--------------------------+----+------+----+--------+---------+---------------+---------+--------------+---------+--------------------+---------+-------------+--------------------+-----+
|电影名|类型|用户名| 评论时间 |用户ID|                      主演|地区|  导演|特色|豆瓣评分|类型Index|        类型Vec|地区Index|       地区Vec|导演Index|             导演Vec|特色Index|      特色Vec|            features|label|
+------+----+------+----------+------+--------------------------+----+------+----+--------+---------+---------------+---------+--------------+---------+--------------------+---------+-------------+--------------------+-----+
|  1028|灾难|对儿勾|2018-01-05|  1576|黄潇|刘翀|杨五洲|王默|肖睿|美国|闫海笑|青春|     8.8|     17.0|(18,[17],[1.0])|      0.0|(20,[0],[1.0])|   4529.0|(12155,[4529],[1.0])|      0.0|(8,[0],[1.0])|(12202,[0,18,19,4...|  8.0|
|  1028|灾难|  Lynn|2018-01-08|  8832|黄潇|刘翀|杨五洲|王默|肖睿|美国|闫海笑|青春|     8.8|     17.0|(18,[17],[1.0])|      0.0|(20,[0],[1.0])|   4529.0|(12155,[4529],[1.0])|      0.0|(8

#### 拆分数据集为训练集和测试集

In [56]:
seed = 42
train_df, test_df = user_movies_df.randomSplit( [0.7, 0.3], seed = seed )

#### Linear Regression 模型

In [57]:
from pyspark.ml.regression import LinearRegression
linreg = LinearRegression(maxIter=500, regParam=0.0)
lm = linreg.fit( train_df )

#### Intercept截距 和 coefficeients系数 的值

In [58]:
lm.intercept

7.260968175237028

In [59]:
lm.coefficients

7.260968175237028

#### 对测试集进行预测

In [60]:
y_pred = lm.transform( test_df )
y_pred.select( 'features', 'label', 'prediction' ).show( 5 )

+--------------------+-----+-----------------+
|            features|label|       prediction|
+--------------------+-----+-----------------+
|(12202,[0,18,19,4...|  8.0|6.241916651012984|
|(12202,[0,18,19,4...|  8.0|6.241916651012984|
|(12202,[0,18,19,4...|  8.0|6.241916651012984|
|(12202,[0,4,20,97...|  8.0|6.371745588268456|
|(12202,[0,4,20,97...|  8.0|6.371745588268456|
+--------------------+-----+-----------------+
only showing top 5 rows



#### 使用RMSE和R-Squared 建模

In [61]:
from pyspark.ml.evaluation import RegressionEvaluator
rmse_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="rmse" )
lm_rmse = rmse_evaluator.evaluate( y_pred )
lm_rmse

2.7182165886282275

In [62]:
r2_evaluator = RegressionEvaluator(labelCol="label",
                            predictionCol="prediction",
                            metricName="r2" )
lm_r2 = r2_evaluator.evaluate( y_pred )
lm_r2

-0.05923048304365497